# Parse similar sentence dataset
- 구글 스프레드시트에서 정리한 simialr sentence 데이터셋을 파싱해서 정리한다
- Requirements
    - `seed_queries.tsv`: seed 쿼리들 정리해놓은 시트의 tsv
    - `pair_labelings.tsv`: candidate pair들 모아서 정리해놓은 시트의 tsv
    - `manual_pair_labelings.tsv`: 매뉴얼하게 pair들 만들어놓은 시트의 tsv

In [1]:
import editdistance

In [17]:
with open("../data/small/test.txt", "r") as f:
    print(sum([1 for line in f]))

15134


In [4]:
import sys
sys.path.append("/home/angrypark/paraphrase_detection/code/")

In [5]:
from utils.utils import JamoProcessor

In [6]:
processor = JamoProcessor()

In [7]:
editdistance.eval(processor.word_to_jamo("그랍시당"), processor.word_to_jamo("노란색만"))

7

In [11]:
import pandas as pd

In [14]:
d = [{"a": 1, "b": 2, "c": 4}, {"a": 1, "b": 2, "c": 3}]

In [15]:
pd.DataFrame(d)

,a,b,c
0,1,2,4
1,1,2,3


In [9]:
len(processor.word_to_jamo("그랍시당"))

12

In [1]:
def parse_seed_queries(seed_query_path):
    '''
    seed query tsv를 파싱함.
    홀수째 row와 짝수째 row가 존댓말 관계이므로, 둘이 1 관계라고 생각하고 이를 파싱한다
    '''
    relations = []
    with open(seed_query_path) as f:
        for line in f.readlines()[1:]:  # first row is header
            splits = line.split('\t')
            for sent1, sent2 in zip(splits[::2], splits[1::2]):
                if len(sent1.strip()) > 0 and len(sent2.strip()) > 0:
                    relations.append((sent1.strip(), sent2.strip(), '1'))
    return relations

In [2]:
parse_seed_queries('dataset/seed_queries.tsv')[:10]

[('나이가 어떻게 돼?', '나이가 어떻게 되세요?', '1'),
 ('갈까?', '갈까요?', '1'),
 ('우리 집에서 라면 먹고 갈래?', '저희 집에서 라면 먹고 가실래요?', '1'),
 ('별로 안 짜증나', '별로 안 짜증나요', '1'),
 ('너 몇살이야?', '몇살이세요?', '1'),
 ('잘잤어?', '잘잤어요?', '1'),
 ('고양이 귀엽지 나도 좋아해', '고양이 귀엽죠 저도 좋아해요', '1'),
 ('나 안 바빠', '저 안 바빠요', '1'),
 ('너는 누구야?', '당신은 누구세요?', '1'),
 ('뭐해?', '뭐해요?', '1')]

In [3]:
import csv
def parse_pair_labelings(pair_labeling_path):
    '''
    pair labeling 된 걸 파싱함.
    tsv 파일이고 (Query, Candidate, Label) 로 되어있다고 생각하자
    '''
    relations = []
    with open(pair_labeling_path) as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter='\t')
        for row in reader:
            # pass empty labels
            label = row['Label'].strip()
            if len(label) > 0 and label not in {'?', '.'}:
                relations.append((row['Query'], row['Candidate'], label))
    return relations

In [4]:
parse_pair_labelings('dataset/manual_pair_labelings.tsv')[:10]

[('너 몇살이야?', '몇살?', '1'),
 ('너 몇살이야?', '몇쨜?', '1'),
 ('잘가', '잘가염', '1'),
 ('잘가', '잘가시죠', '1'),
 ('잘가', '빨리 가', '0'),
 ('잘가', '잘까', '0'),
 ('잘가', '잘까요?', '0'),
 ('잘가', '잘갔어?', '0'),
 ('잘가', '잘 도착했어?', '0'),
 ('안녕?', '안녕이라고 말하지마', '0')]

In [5]:
all_relations = \
    parse_seed_queries('dataset/seed_queries.tsv') \
    + parse_pair_labelings('dataset/pair_labelings.tsv') \
    + parse_pair_labelings('dataset/manual_pair_labelings.tsv')
    
len(all_relations)

9566

In [6]:
"""
A union-find disjoint set data structure.
"""

# 2to3 sanity
from __future__ import (
    absolute_import, division, print_function, unicode_literals,
)

# Third-party libraries
import numpy as np


class UnionFind(object):
    """Union-find disjoint sets datastructure.
    Union-find is a data structure that maintains disjoint set
    (called connected components or components in short) membership,
    and makes it easier to merge (union) two components, and to find
    if two elements are connected (i.e., belong to the same
    component).
    This implements the "weighted-quick-union-with-path-compression"
    union-find algorithm.  Only works if elements are immutable
    objects.
    Worst case for union and find: :math:`(N + M \log^* N)`, with
    :math:`N` elements and :math:`M` unions. The function
    :math:`\log^*` is the number of times needed to take :math:`\log`
    of a number until reaching 1. In practice, the amortized cost of
    each operation is nearly linear [1]_.
    Terms
    -----
    Component
        Elements belonging to the same disjoint set
    Connected
        Two elements are connected if they belong to the same component.
    Union
        The operation where two components are merged into one.
    Root
        An internal representative of a disjoint set.
    Find
        The operation to find the root of a disjoint set.
    Parameters
    ----------
    elements : NoneType or container, optional, default: None
        The initial list of elements.
    Attributes
    ----------
    n_elts : int
        Number of elements.
    n_comps : int
        Number of distjoint sets or components.
    Implements
    ----------
    __len__
        Calling ``len(uf)`` (where ``uf`` is an instance of ``UnionFind``)
        returns the number of elements.
    __contains__
        For ``uf`` an instance of ``UnionFind`` and ``x`` an immutable object,
        ``x in uf`` returns ``True`` if ``x`` is an element in ``uf``.
    __getitem__
        For ``uf`` an instance of ``UnionFind`` and ``i`` an integer,
        ``res = uf[i]`` returns the element stored in the ``i``-th index.
        If ``i`` is not a valid index an ``IndexError`` is raised.
    __setitem__
        For ``uf`` and instance of ``UnionFind``, ``i`` an integer and ``x``
        an immutable object, ``uf[i] = x`` changes the element stored at the
        ``i``-th index. If ``i`` is not a valid index an ``IndexError`` is
        raised.
    .. [1] http://algs4.cs.princeton.edu/lectures/
    """

    def __init__(self, elements=None):
        self.n_elts = 0  # current num of elements
        self.n_comps = 0  # the number of disjoint sets or components
        self._next = 0  # next available id
        self._elts = []  # the elements
        self._indx = {}  #  dict mapping elt -> index in _elts
        self._par = []  # parent: for the internal tree structure
        self._siz = []  # size of the component - correct only for roots

        if elements is None:
            elements = []
        for elt in elements:
            self.add(elt)


    def __repr__(self):
        return  (
            '<UnionFind:\n\telts={},\n\tsiz={},\n\tpar={},\nn_elts={},n_comps={}>'
            .format(
                self._elts,
                self._siz,
                self._par,
                self.n_elts,
                self.n_comps,
            ))

    def __len__(self):
        return self.n_elts

    def __contains__(self, x):
        return x in self._indx

    def __getitem__(self, index):
        if index < 0 or index >= self._next:
            raise IndexError('index {} is out of bound'.format(index))
        return self._elts[index]

    def __setitem__(self, index, x):
        if index < 0 or index >= self._next:
            raise IndexError('index {} is out of bound'.format(index))
        self._elts[index] = x

    def add(self, x):
        """Add a single disjoint element.
        Parameters
        ----------
        x : immutable object
        Returns
        -------
        None
        """
        if x in self:
            return
        self._elts.append(x)
        self._indx[x] = self._next
        self._par.append(self._next)
        self._siz.append(1)
        self._next += 1
        self.n_elts += 1
        self.n_comps += 1

    def find(self, x):
        """Find the root of the disjoint set containing the given element.
        Parameters
        ----------
        x : immutable object
        Returns
        -------
        int
            The (index of the) root.
        Raises
        ------
        ValueError
            If the given element is not found.
        """
        if x not in self._indx:
            raise ValueError('{} is not an element'.format(x))

        p = self._indx[x]
        while p != self._par[p]:
            # path compression
            q = self._par[p]
            self._par[p] = self._par[q]
            p = q
        return p

    def connected(self, x, y):
        """Return whether the two given elements belong to the same component.
        Parameters
        ----------
        x : immutable object
        y : immutable object
        Returns
        -------
        bool
            True if x and y are connected, false otherwise.
        """
        return self.find(x) == self.find(y)

    def union(self, x, y):
        """Merge the components of the two given elements into one.
        Parameters
        ----------
        x : immutable object
        y : immutable object
        Returns
        -------
        None
        """
        # Initialize if they are not already in the collection
        for elt in [x, y]:
            if elt not in self:
                self.add(elt)

        xroot = self.find(x)
        yroot = self.find(y)
        if xroot == yroot:
            return
        if self._siz[xroot] < self._siz[yroot]:
            self._par[xroot] = yroot
            self._siz[yroot] += self._siz[xroot]
        else:
            self._par[yroot] = xroot
            self._siz[xroot] += self._siz[yroot]
        self.n_comps -= 1

    def component(self, x):
        """Find the connected component containing the given element.
        Parameters
        ----------
        x : immutable object
        Returns
        -------
        set
        Raises
        ------
        ValueError
            If the given element is not found.
        """
        if x not in self:
            raise ValueError('{} is not an element'.format(x))
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        return set(elts[roots == self.find(x)])

    def components(self):
        """Return the list of connected components.
        Returns
        -------
        list
            A list of sets.
        """
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        distinct_roots = set(roots)
        return [set(elts[roots == root]) for root in distinct_roots]
        # comps = []
        # for root in distinct_roots:
        #     mask = (roots == root)
        #     comp = set(elts[mask])
        #     comps.append(comp)
        # return comps

    def component_mapping(self):
        """Return a dict mapping elements to their components.
        The returned dict has the following semantics:
            `elt -> component containing elt`
        If x, y belong to the same component, the comp(x) and comp(y)
        are the same objects (i.e., share the same reference). Changing
        comp(x) will reflect in comp(y).  This is done to reduce
        memory.
        But this behaviour should not be relied on.  There may be
        inconsitency arising from such assumptions or lack thereof.
        If you want to do any operation on these sets, use caution.
        For example, instead of
        ::
            s = uf.component_mapping()[item]
            s.add(stuff)
            # This will have side effect in other sets
        do
        ::
            s = set(uf.component_mapping()[item]) # or
            s = uf.component_mapping()[item].copy()
            s.add(stuff)
        or
        ::
            s = uf.component_mapping()[item]
            s = s | {stuff}  # Now s is different
        Returns
        -------
        dict
            A dict with the semantics: `elt -> component contianing elt`.
        """
        elts = np.array(self._elts)
        vfind = np.vectorize(self.find)
        roots = vfind(elts)
        distinct_roots = set(roots)
        comps = {}
        for root in distinct_roots:
            mask = (roots == root)
            comp = set(elts[mask])
            comps.update({x: comp for x in comp})
            # Change ^this^, if you want a different behaviour:
            # If you don't want to share the same set to different keys:
            # comps.update({x: set(comp) for x in comp})
        return comps

In [7]:
def create_node_set_from_relations(relations):
    unique_strings = set()
    for s1, s2, _ in relations:
        unique_strings.add(s1)
        unique_strings.add(s2)
    unique_strings = list(unique_strings)
    print(len(unique_strings))
    
    uf = UnionFind(unique_strings)
    for s1, s2, r in relations:
        if r == '1':
            uf.union(s1, s2)
        
    return uf

uf = create_node_set_from_relations(all_relations)

9508


In [8]:
from itertools import permutations

def mine_all_relations(relations, union_find):
    '''
    relation과 node set으로부터 가능한 모든 pair labeling들을 만들어낸다
    만들 수 있는 케이스들은
    (1) node set 내의 string들끼리 묶은 것 nP2개
    (2) <나 >에서 한쪽 string을 node set의 것으로 치환한 것 2(n1 n2)개
    (3) 0 relation에서 한쪽만 node set의 것으로 치환한것 2(n1+n2)개
    '''
    def neg_r(r):
        assert r in {'<', '>'}
        return '>' if r == '<' else '<'
    
    all_relation_set = set()
    
    # 일단 node set들을 돌며 nP2개를 만들자
    node_sets = union_find.components()
    for nodes in node_sets:
        for n1, n2 in permutations(nodes, 2):
            all_relation_set.add((n1, n2, '1'))
    
    # 이제 relation들을 돌며 케이스들에 맞게 add하자
    node_set_mappings = uf.component_mapping()
    for string1, string2, r in relations:
        # '>' 이나 '<' 인 경우
        if r in {'>', '<'}:
            for n1 in node_set_mappings[string1]:
                for n2 in node_set_mappings[string2]:
                    all_relation_set.add((n1, n2, r))
                    all_relation_set.add((n2, n1, neg_r(r)))
        # '0' 인 경우
        elif r == '0':
            for n1 in node_set_mappings[string1]:
                all_relation_set.add((n1, string2, '0'))
                all_relation_set.add((string2, n1, '0'))
            for n2 in node_set_mappings[string2]:
                all_relation_set.add((string1, n2, '0'))
                all_relation_set.add((n2, string1, '0'))
                
    return all_relation_set


In [9]:
all_relation_set = mine_all_relations(all_relations, uf)

In [10]:
# check if there is double relation
from collections import defaultdict
pair_relations = defaultdict(list)

for s1, s2, r in all_relation_set:
    pair_relations[s1, s2].append(r)
pair_relations = dict(pair_relations)

In [11]:
from collections import Counter
Counter(len(x) for x in pair_relations.values())

Counter({1: 217004})

### Train/Test split

In [12]:
# all relation set을 돌면서 node set 사이의 interaction 유무를 보고, 그걸로 forest를 만들어보자
nodes = set()
edges = {}

for s1, s2, r in all_relation_set:
    s1_node_set_idx = uf.find(s1)
    s2_node_set_idx = uf.find(s2)
    nodes.add(s1_node_set_idx)
    nodes.add(s2_node_set_idx)
    
    edges[s1_node_set_idx, s2_node_set_idx] = r

In [13]:
node_set_idxs = sorted(list(nodes))

In [14]:
node_set_uf = UnionFind(node_set_idxs)
for n1, n2 in edges.keys():
    node_set_uf.union(n1, n2)

In [15]:
len(node_set_uf.components())

230

In [16]:
# 230개의 forest가 생긴다.
# 이걸 베이스로 random하게 forest들을 train/val/test (8:1:1) 로 split하고, 이걸로 pair들을 split하자

In [24]:
import random
random.seed(3091)

node_set_idx_to_tvt = {}
for components in node_set_uf.components():
    random_value = random.random()
    if random_value < 0.8:
        tvt = 'train'
    elif random_value < 0.9:
        tvt = 'val'
    else:
        tvt = 'test'
        
    for c in components:
        node_set_idx_to_tvt[c] = tvt

In [25]:
pairs = {
    'train': list(),
    'val': list(),
    'test': list()
}

for s1, s2, r in all_relation_set:
    s1_node_set_idx = uf.find(s1)
    s2_node_set_idx = uf.find(s2)
    assert node_set_idx_to_tvt[s1_node_set_idx] == node_set_idx_to_tvt[s2_node_set_idx]
    
    tvt = node_set_idx_to_tvt[s1_node_set_idx]
    pairs[tvt].append((s1, s2, r))

In [26]:
{k: len(v) for k, v in pairs.items()}

{'test': 15134, 'train': 182558, 'val': 19312}

In [27]:
# 이제 train set에서 원활한 학습을 위해 random negative들을 만들자.
# train set에 있는 string 2개를 random하게 sampling해서 random negative 들을 만들것.
train_random_negatives = []
number_of_negatives_to_generate = 300000 - len(pairs['train'])

train_strings = set()
for s1, s2, r in pairs['train']:
    train_strings.add(s1)
    train_strings.add(s2)
train_strings = sorted(list(train_strings))

continue_count = 0
while len(train_random_negatives) < number_of_negatives_to_generate:
    string1 = random.choice(train_strings)
    string2 = random.choice(train_strings)
    
    string1_nodeset = uf.find(string1)
    string2_nodeset = uf.find(string2)
    
    # node set이 같거나, 기존에 interaction이 있으면 pass
    if string1_nodeset == string2_nodeset:
        continue
    if (string1_nodeset, string2_nodeset) in edges or (string2_nodeset, string1_nodeset) in edges:
        continue_count += 1
        continue
        
    train_random_negatives.append((string1, string2, '0'))

In [28]:
train_random_negatives[:20]

[('헤헤 헹', '취미 있으신가요?', '0'),
 ('나 왔어 왔어', '비 왔었어', '0'),
 ('흐 다행이네요', '툭툭', '0'),
 ('일어나서 밥 먹음', '드시러 오실래여?', '0'),
 ('벌써 한시반이네요', '웅 지웠어요', '0'),
 ('큭 머하고이써?', '축하 해', '0'),
 ('추리닝만으로 확실합니까?', '어떻게 갔다 드려요?', '0'),
 ('내 얘기좀 해', '어 화났어?', '0'),
 ('이제 자러 간다', '근데 이렇게 연락하게 되네요', '0'),
 ('비가 한방울씩 떨어지네요', '말도 안되는 소리 하지 말아요', '0'),
 ('질문해줘요', '왜 자꾸 거짓말해?', '0'),
 ('흐헝 아니요', '외로운게 아니고', '0'),
 ('받고싶은거 없어요?', '내물음에 대답좀 해주세요', '0'),
 ('한 눈치 하나봐요?', '예리이고 싶네', '0'),
 ('너는 무슨 음악좋아해?', '오늘 영화보러 갈거양', '0'),
 ('지금 바쁘냐?', '축 축하해', '0'),
 ('언제나 치킨은 맛있어요', '장난입니다?', '0'),
 ('오늘은 원치않은 외출들이었어요', '뭐하구 있어요?', '0'),
 ('아프지 마라', '주로 어떤 노래 부르세여?', '0'),
 ('와 대박', '그 그래라', '0')]

In [29]:
pairs['train'].extend(train_random_negatives)

In [30]:
def write_pair_to_txt(pair_list, path):
    with open(path, 'w') as f:
        for s1, s2, r in pair_list:
            f.write('{}\t{}\t{}\n'.format(s1, s2, r))
            
import os
base_dir = '/home/shuuki4/sandbox_project/similar_sentence/simsent_data'

write_pair_to_txt(
    pairs['train'], os.path.join(base_dir, 'train.txt'))
write_pair_to_txt(
    pairs['val'], os.path.join(base_dir, 'val.txt'))
write_pair_to_txt(
    pairs['test'], os.path.join(base_dir, 'test.txt'))